In [ ]:
import os
import pickle

import keras.optimizers
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.callbacks import ModelCheckpoint
import pandas as pd

In [ ]:
input_path = 'data'
preparation_path = os.path.join(input_path, '02_preparation')
model_path = os.path.join(input_path, '03_model')
checkpoint_path = os.path.join(model_path, 'lstm', 'checkpoints')

In [ ]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(130))
    model.add(Activation('softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.00002)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    return model

In [ ]:
model = lstm_model()

mc = ModelCheckpoint(os.path.join(checkpoint_path, 'best_model.h5'), monitor='val_loss', mode='min',
                     save_best_only=True, verbose=1)

file = os.path.join(preparation_path, f'x_tr.pkl')
with open(file, 'rb') as f:
    x_tr = pickle.load(f).astype(np.float32)

file = os.path.join(preparation_path, f'x_val.pkl')
with open(file, 'rb') as f:
    x_val = pickle.load(f).astype(np.float32)

file = os.path.join(preparation_path, f'y_tr.pkl')
with open(file, 'rb') as f:
    y_tr = pickle.load(f).astype(np.float32)

file = os.path.join(preparation_path, f'y_val.pkl')
with open(file, 'rb') as f:
    y_val = pickle.load(f).astype(np.float32)

x_tr = x_tr.reshape(-1, 1, 32)
y_tr = y_tr.reshape(-1, 1)
x_val = x_val.reshape(-1, 1, 32)
y_val = y_val.reshape(-1, 1)

history = model.fit(np.array(x_tr), np.array(y_tr), batch_size=128, epochs=3000, validation_data=(np.array(x_val),
                                                                                                  np.array(y_val)),
                    verbose=1, callbacks=[mc])

model_history = pd.DataFrame(history.history)
model_history['epoch'] = history.epoch

model_history.to_csv('model_history.csv', index=False)
